## 高性能爬虫实现爬虫
通过前面的内容，我们已经能够完成一个爬虫，但是这个时候程序都是单线程，速度很慢。如何实现更高效的爬虫？

- 多线程的方式实现爬虫
- 多进程的方式实现爬虫
- 线程池的方式实现爬虫
- 协程池的方式实现爬虫

### 爬取糗事百科段子
#### 单线程爬虫实现

**糗事百科地址：https://www.qiushibaike.com/**

In [14]:
import requests
from lxml import etree

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
}
url = 'https://www.qiushibaike.com/8hr/page/3/'
urllist = ['https://www.qiushibaike.com/8hr/page/{}/'.format(i) for i in range(1,2)]
for url in urllist:
    res = requests.get(url, headers=headers)
    htmlstr = res.content.decode()
    html = etree.HTML(htmlstr)
    lilist = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
#     print(lilist)
    contentlist = []
    for li in lilist:
        item = {}
        item['username'] = li.xpath('./div/div/a/span/text()')[0].strip()
#         print(item)
        item['content'] = li.xpath('./div/a/text()')
        contentlist.append(item)
contentlist

[{'username': '窝里斗窝...', 'content': ['今天喂点儿好吃的']},
 {'username': '一枕清霜', 'content': ['有本事你冲我来啊']},
 {'username': '欧洲血统...', 'content': ['？？？']},
 {'username': '倾心倾情...',
  'content': ['每次午休，我们关上门，boss就来敲门说事，后来就不午休了，可是不午休了吧boss还倒是不来了，再后来……我们开着门午休……[捂脸]']},
 {'username': '逗b聚乐...', 'content': ['流氓界的清流']},
 {'username': 'blue...', 'content': ['果然女孩子生起气来真的是不分年龄，都是一样的。']},
 {'username': '土豆se...', 'content': ['米切尔最后时刻致命失误！葬送了比赛啊！还是太年轻了！']},
 {'username': '逗b聚乐...', 'content': ['俗话说：']},
 {'username': '一口吳', 'content': ['来自小柴l的早安问候~✧*。٩و✧*。希望你今天心里住进小太阳']}]

In [15]:
contentlist

[{'username': '窝里斗窝...', 'content': ['今天喂点儿好吃的']},
 {'username': '一枕清霜', 'content': ['有本事你冲我来啊']},
 {'username': '欧洲血统...', 'content': ['？？？']},
 {'username': '倾心倾情...',
  'content': ['每次午休，我们关上门，boss就来敲门说事，后来就不午休了，可是不午休了吧boss还倒是不来了，再后来……我们开着门午休……[捂脸]']},
 {'username': '逗b聚乐...', 'content': ['流氓界的清流']},
 {'username': 'blue...', 'content': ['果然女孩子生起气来真的是不分年龄，都是一样的。']},
 {'username': '土豆se...', 'content': ['米切尔最后时刻致命失误！葬送了比赛啊！还是太年轻了！']},
 {'username': '逗b聚乐...', 'content': ['俗话说：']},
 {'username': '一口吳', 'content': ['来自小柴l的早安问候~✧*。٩و✧*。希望你今天心里住进小太阳']}]

In [18]:
import requests
from lxml import etree
import time


class QiuBai():
    def __init__(self):
        self.tmp_url = 'https://www.qiushibaike.com/8hr/page/{}/'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }
    """准备url地址的列表"""
    def get_url_list(self):
        return [self.tmp_url.format(i) for i in range(1,14)]
    """发送请求 获取响应"""
    def parse_url(self, url):
        res = requests.get(url, headers=self.headers)
        return res.content.decode()
    """提取数据"""
    def get_content(self, htmlstr):
        html = etree.HTML(htmlstr)
        lilist = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
        contentlist = []
        for li in lilist:
            item = {}
            item['username'] = li.xpath('./div/div/a/span/text()')[0].strip()
            item['content'] = li.xpath('./div/a/text()')
            contentlist.append(item)
        return contentlist
    """保存数据"""
    def save_content(self, contentlist):
        for content in contentlist:
            print(content)
    """爬虫的主要逻辑"""
    def run(self):
        # 1 准备URL列表
        urllist = self.get_url_list()
        # 2 遍历URLlist 发送请求 获取响应 
        for url in urllist:
            htmlstr = self.parse_url(url)
            # 3 提取数据
            contentlist = self.get_content(htmlstr)
            # 4 保存
            self.save_content(contentlist)

if __name__== '__main__':
    start = time.time()
    qiubai = QiuBai()
    qiubai.run()
    print(f"耗时：{time.time()-start}")

{'username': '名字太俊...', 'content': ['我准备以后不让他俩来参加活动了！']}
{'username': '撩汉大婶', 'content': ['此文采是多么盲流的风采']}
{'username': '给叮叮把...', 'content': ['什么是神回复……请看图……']}
{'username': '石锅拌着...', 'content': ['说个傻人有傻福的故事。有一发小人称莽哥，莽哥也有名字的，只因他莽出了境界，莽的有盐有味，莽的十里八乡都知道这个名字，故而再没人记得他本来叫什么。刚刚出生没多久父']}
{'username': '-奶酪酱...', 'content': ['究竟是掉色了，还是生着生着没墨了？']}
{'username': '一大坨宠...', 'content': ['“你这是什么意思？”']}
{'username': '知-识-...', 'content': ['十二、偶有……']}
{'username': '一枕清霜', 'content': ['经历了8个月的假期，送走孩子，父母们终于能放松一下了']}
{'username': '老头子1...', 'content': ['在糗百上认识了一个女网友，我们聊了几个星期，感觉挺好的我就约她，她开始有点犹豫，后面还是同意了。她说了她的性格，我们也互换了照片，她说她颜值一般，本以为会是大型']}
{'username': '凯德无', 'content': ['消化不了只能辞职']}
{'username': '、(り执...', 'content': ['影视剪辑系列11.']}
{'username': '王保平o...', 'content': ['荷兰摄影师DDu在维也纳旅行时，捕捉到了这只地松鼠和一朵小黄花之间不可磨灭的自然联系。']}
{'username': '零點糗事', 'content': ['自己踩自己脚的舞步']}
{'username': '白玉无尘', 'content': ['万万没想到啊']}
{'username': '加州不老...', 'content': ['本以为是举手之劳，原来是多此一举，自取其辱！']}
{'username': '陀舍幻宇', 'content': ['想看看你们的聊天记录

{'username': 'hist...', 'content': ['昨天奔驰最大的争议吧。有人说不够气派，可我决定先追随再理解']}
{'username': '少时人间...', 'content': ['请问在哪里能买到您的名著呢？']}
{'username': 'Zyqq...', 'content': ['打不到！诶诶～打～不～到～']}
{'username': '勇敢的坚...', 'content': ['换我炒菜了']}
{'username': '缘来～是...', 'content': ['脑瓜子嗡嗡的吧？']}
{'username': '昕庭～', 'content': ['睡前小故事：气息“哎呀，怎么没用啊！”女孩双手掐着法诀，不停的抵挡着妖怪的攻击，但从前无往不利的术式打在妖怪身上竟然不痛不痒，没一会儿，女孩竟然已被逼入了绝境。']}
{'username': '禹扬木美', 'content': ['有一程序员在网上晒出了与领导吃饭时的照片。']}
{'username': '机智的猴...', 'content': ['刚开始的我，和被生活摧残后的我']}
{'username': '宋单单9...', 'content': ['口袋里的小狗。']}
{'username': '这个和尚...', 'content': ['两全其美的好办法']}
{'username': '小妞我是...', 'content': ['莺哥晒新造型～可与杜姓模版绝代双骄一帅否']}
{'username': '凯德无', 'content': ['消化不了只能辞职']}
耗时：1.0014338493347168


#### 多线程爬虫实现

##### 多线程的方法使用
- 在python3中，主线程主进程结束，子线程，子进程不会结束
- 为了能够让主线程回收子线程，可以把子线程设置为守护线程，即该线程不重要，主线程结束，子线程结束
```
t1 = threading.Thread(targe=func,args=(,))
t1.setDaemon(True) # 设置为守护线程
t1.start() #此时线程才会启动
```

##### 队列模块的使用
```
from queue import Queue
q = Queue(maxsize=100)  # maxsize为队列长度
item = {}
q.put_nowait(item)  # 不等待直接放，队列满的时候会报错
q.put(item)  # 放入数据，队列满的时候会阻塞等待
q.get_nowait()  # 不等待直接取，队列空的时候会报错
q.get()  # 取出数据，队列为空的时候会阻塞等待
q.qsize()  # 获取队列中现存数据的个数
q.join()  # 队列中维持了一个计数(初始为0)，计数不为0时候让主线程阻塞等待，队列计数为0的时候才会继续往后执行
            # q.join()实际作用就是阻塞主线程，与task_done()配合使用
            # put()操作会让计数+1，task_done()会让计数-1
            # 计数为0，才停止阻塞，让主线程继续执行
q.task_done()  # put的时候计数+1，get不会-1，get需要和task_done 一起使用才会-1
```

##### 多线程实现思路剖析
- 把爬虫中的每个步骤封装成函数，分别用线程去执行
- 不同的函数通过队列相互通信，函数间解耦

<img src="http://39.106.195.80:8888/group1/M00/00/00/wKgCa1_RzOKAU4BJAANe2BQKxJ8507.png" width=90%>

In [25]:
import requests
from lxml import etree
import time
from queue import Queue
import threading


class MTQiuBai():
    def __init__(self):
        self.tmp_url = 'https://www.qiushibaike.com/8hr/page/{}/'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }
        self.url_queue = Queue()
        self.html_queue = Queue()
        self.contentlist_queue = Queue()
    """准备url地址的列表"""

    def get_url_list(self):
        for i in range(1, 14):
            self.url_queue.put(self.tmp_url.format(i))

    """发送请求 获取响应"""

    def parse_url(self):
        while True:
            url = self.url_queue.get()
            res = requests.get(url, headers=self.headers)
            self.html_queue.put(res.content.decode())
            self.url_queue.task_done()

    """提取数据"""

    def get_content(self):
        while True:
            htmlstr = self.html_queue.get()
            html = etree.HTML(htmlstr)
            lilist = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
            contentlist = []
            for li in lilist:
                item = {}
                item['username'] = li.xpath('./div/div/a/span/text()')[0].strip()
                item['content'] = li.xpath('./div/a/text()')
                contentlist.append(item)
            self.contentlist_queue.put(contentlist)
            self.html_queue.task_done()

    """保存数据"""

    def save_content(self):
        while True:
            contentlist = self.contentlist_queue.get()
            for content in contentlist:
                print(content)  # 做一个数据保存操作
            self.contentlist_queue.task_done()
    """爬虫的主要逻辑"""

    def run(self):
        thread_list = []
        # 1 准备URL列表
        t_url = threading.Thread(target=self.get_url_list)
        thread_list.append(t_url)

        # 2 遍历URLlist 发送请求 获取响应
        for i in range(3):  # 3个线程发送请求
            t_parse = threading.Thread(target=self.parse_url)
            thread_list.append(t_parse)

        t_content = threading.Thread(target=self.get_content)
        thread_list.append(t_content)

        t_save = threading.Thread(target=self.save_content)
        thread_list.append(t_save)

        for i in thread_list:
            i.setDaemon(True)  # 把子线程设置为守护线程
            i.start()

        for q in [self.url_queue, self.html_queue, self.contentlist_queue]:
            q.join()  # 让主线程阻塞， 等待队列的计数为0
        print('主线程结束')


if __name__ == '__main__':
    start = time.time()
    mtqiubai = MTQiuBai()
    mtqiubai.run()
    print(f"耗时：{time.time()-start}")

{'username': '小猴子(...', 'content': ['爆笑萌娃，这表情简直太魔性了']}
{'username': '白玉无尘', 'content': ['闺蜜那些扎心的话']}
{'username': '知-识-...', 'content': ['八、前几……']}
{'username': '满天繁星...', 'content': ['说个我高中时候的事，我们文科，男少女多，我们小组只有两个男生，每次打扫卫生什么重活那小子都指着我干，不动手，感觉自己很帅似的。说好我俩涮洗，女生拖地，我把两个拖']}
{'username': '凯枫的围...', 'content': ['？？？']}
{'username': '带翅膀的...', 'content': ['正在超市闲逛，忽然听到超市里的广播喊到:请张家林的家长马上到前台来认领你的孩子！消息一出，只见一对年轻的夫妇急冲冲地往前台赶，想必那是他们的孩子。忽然，那女拉住']}
{'username': '少爷杨姐...', 'content': ['面粉：你有事吗？']}
{'username': '梦dem...', 'content': ['啊？原来是——']}
{'username': '今天困困...', 'content': ['如何避免下快攻时被詹姆斯追帽？请参考图二，可以说是教科书般了！']}
{'username': '九头纪', 'content': ['某日，俺们初中的语文老师站在讲台上拿着点名册突然对俺们说：“你们班的姓氏好丰盛啊！”俺们：“啥？”老师：“有米(芈)有饭（范）有菜（蔡）有盐(严)有糖(唐)有粥']}
{'username': '笑傲小姐...', 'content': ['男孩和网恋半年的女友奔现后，自己的胳膊都快被咬成五环了……']}
{'username': '土豆se...', 'content': ['这个笑死我了，保罗因为系鞋带时间过久吃T']}
{'username': '带翅膀的...', 'content': ['一大爷拿卡去银行取1000元；营业员：大爷，3000元以下请到自动取款机上取。大爷：我不会用啊！营业员：3000元以下柜台不办理的。大爷好话说了半天营业员还是不']}
{'username': '哇伊伊', 'content

注意点：
put会让队列的计数+1，但是单纯的使用get不会让其-1，需要和task_done同时使用才能够-1
task_done不能放在另一个队列的put之前，否则可能会出现数据没有处理完成，程序结束的情况

#### 多进程爬虫实现

##### 多进程程的方法使用
```
from multiprocessing import Process  # 导入模块
t1 = Process(targe=func, args=(,))  # 使用一个进程来执行一个函数
t1.daemon = True  # 设置为守护进程
t1.start()  # 此时线程才会启动
```
##### 多进程中队列的使用
- 多进程中使用普通的队列模块会发生阻塞，对应的需要使用multiprocessing提供的JoinableQueue模块，其使用过程和在线程中使用的queue方法相同

In [26]:
import requests
from lxml import etree
from multiprocessing import Process
from multiprocessing import JoinableQueue as Queue


class MPQiuBai:
    def __init__(self):
        self.url_temp = "https://www.qiushibaike.com/8hr/page/{}/"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"
        }
        self.url_queue = Queue()  # 保存url
        self.html_queue = Queue()  # 保存html字符串
        self.content_queue = Queue()  # 保存提取到的数据

    def get_url_list(self):
        for i in range(1, 14):
            self.url_queue.put(self.url_temp.format(i))

    def parse_url(self):
        while True:
            url = self.url_queue.get()
            html_str = requests.get(url, headers=self.headers).content.decode()
            self.html_queue.put(html_str)
            self.url_queue.task_done()

    def get_content_list(self):
        while True:
            html_str = self.html_queue.get()
            html = etree.HTML(html_str)
            div_list = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
            content_list = []
            for div in div_list:
                content = {}
                content["user_name"] = div.xpath(
                    "./div/div/a/span/text()")[0].strip()
                content["content"] = [i.strip()
                                      for i in div.xpath("./div/a/text()")]
                content_list.append(content)
            self.content_queue.put(content_list)
            self.html_queue.task_done()

    def save_content_list(self):
        while True:
            content_list = self.content_queue.get()
            for content in content_list:
                print(content)  # 此处对数据进行保存操作
            self.content_queue.task_done()

    def run(self):
        process_list = []
        # 1. url_list
        t_url = Process(target=self.get_url_list)
        process_list.append(t_url)
        # 2. 遍历，发送请求
        for i in range(3):  # 创建3个子进程
            t_parse = Process(target=self.parse_url)
            process_list.append(t_parse)
        # 3. 提取数据
        t_content = Process(target=self.get_content_list)
        process_list.append(t_content)
        # 4. 保存
        t_save = Process(target=self.save_content_list)
        process_list.append(t_save)

        for t in process_list:
            t.daemon = True  # 把进线程设置为守护线程，主进程技术，子进程结束
            t.start()

        for q in [self.url_queue, self.html_queue, self.content_queue]:
            q.join()  # 让主进程阻塞

        print("主进程结束")


if __name__ == '__main__':
    t1 = time.time()
    mp_qiubai = MPQiuBai()
    mp_qiubai.run()
    print("total cost:", time.time()-t1)

{'user_name': '哇伊伊', 'content': ['一顿操作猛如虎，一看工资两千五']}
{'user_name': '白玉无尘', 'content': ['嫂子回娘家了，好无聊啊']}
{'user_name': '爱运动的...', 'content': ['皮尔斯更推：我看好掘金拿下抢七?????????我敢赌上我的房子！']}
{'user_name': '18cc...', 'content': ['假如生活欺騙了你，不要急，後面還有很多被騙的時候']}
{'user_name': '运城王菲', 'content': ['玩游戏时候遇到的，哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈这个刘阿斗太可爱了！']}
{'user_name': '飞漠刀影...', 'content': ['昨晚照常加完班回家，有一段路是没有路灯的，突然听见一个熟悉的声音说你先起来，哎呀……是我爸的声音，然后有个陌生的女声在嚷，哎呀你先别动，让我先动什么的。卧槽？大']}
{'user_name': '初めまし...', 'content': ['粗鄙之语，哈哈哈']}
{'user_name': 'Q小飞', 'content': ['天上挂着一只大猪蹄子！']}
{'user_name': '而是共同...', 'content': ['搬砖日记！']}
{'user_name': '狐小乔', 'content': ['老爸和老妈吵架了，两个人冷战。作为家里的贴心小棉袄外加开心果，我义不容辞的承担了缓解家庭矛盾的责任。我蹭到老妈旁边，乖巧地说:“妈，以后我不结婚，就养条狗，孝敬']}
{'user_name': 'Kaha...', 'content': ['原谅我不厚道得笑了']}
{'user_name': '林天宏l...', 'content': ['被这个画面莫名戳到了~']}
{'user_name': '幼稚老阿...', 'content': ['帮我朋友问的，一个女的因为丑，找不到男朋友，那我该，不是，那我朋友该怎么办']}
{'user_name': '可口可乐...', 'content': ['问君能有几多愁，恰似饭桌卡狗头']}
{'user_name': 'Cc_喵...', 'content': ['这手刹，给力']}
{

上述多进程实现的代码中，multiprocessing提供的JoinableQueue可以创建可连接的共享进程队列。和普通的Queue对象一样，队列允许项目的使用者通知生产者项目已经被成功处理。通知进程是使用共享的信号和条件变量来实现的。 对应的该队列能够和普通队列一样能够调用task_done和join方法

#### 线程池实现爬虫

##### 实例化线程池对象
```
from multiprocessing.dummy import Pool
pool = Pool(processes=3)  # 默认大小是cup的个数
"""源码内容：
if processes is None:
     processes = os.cpu_count() or 1
     # 此处or的用法：
     默认选择or前边的值，
     如果or前边的值为False，就选择后边的值
"""
```

##### 把从发送请求，提取数据，到保存合并成一个函数，交给线程池异步执行
使用方法pool.apply_async(func)
```
def exetute_requests_item_save(self):
    url = self.queue.get()
    html_str = self.parse_url(url)
    content_list = self.get_content_list(html_str)
    self.save_content_list(content_list)
    self.total_response_num += 1
    
pool.apply_async(self.exetute_requests_item_save)
```

##### 添加回调函数

- 通过apply_async的方法能够让函数异步执行，但是只能够执行一次
- 为了让其能够被反复执行，通过添加回调函数的方式能够让_callback 递归的调用自己
- 同时需要指定递归退出的条件

```
def _callback(self, temp):
     if self.is_running:
          pool.apply_async(self.exetute_requests_item_save,
                           callback=self._callback)

 pool.apply_async(self.exetute_requests_item_save,callback=self._callback)
```

##### 确定程序结束的条件 程序在获取的响应和url数量相同的时候可以结束
```
while True:  # 防止主线程结束
     time.sleep(0.0001)  # 避免cpu空转，浪费资源
     if self.total_response_num >= self.total_requests_num:
         self.is_running = False
         break
 self.pool.close() #关闭线程池，防止新的线程开启
# self.pool.join() #等待所有的子线程结束
```

In [27]:
import requests
from lxml import etree
from queue import Queue
from multiprocessing.dummy import Pool
import time


class TQQiuBai:
    def __init__(self):
        self.url_temp = "https://www.qiushibaike.com/8hr/page/{}/"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"
        }
        self.queue = Queue()
        self.pool = Pool(5)
        self.is_running = True
        self.total_requests_num = 0
        self.total_response_num = 0

    def get_url_list(self):  # 获取url列表
        for i in range(1, 14):
            self.queue.put(self.url_temp.format(i))
            self.total_requests_num += 1

    def parse_url(self, url):  # 发送请求，获取响应
        return requests.get(url, headers=self.headers).content.decode()

    def get_content_list(self, html_str):  # 提取段子
        html = etree.HTML(html_str)
        div_list = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
        content_list = []
        for div in div_list:
            content = {}
            content["user_name"] = div.xpath(
                "./div/div/a/span/text()")[0].strip()
            content["content"] = [i.strip()
                                  for i in div.xpath("./div/a/text()")]
            content_list.append(content)
        return content_list

    def save_content_list(self, content_list):  # 保存数据
        for content in content_list:
            print(content)  # 此处对数据进行保存操作

    def exetute_requests_item_save(self):
        url = self.queue.get()
        html_str = self.parse_url(url)
        content_list = self.get_content_list(html_str)
        self.save_content_list(content_list)
        self.total_response_num += 1

    def _callback(self, temp):
        if self.is_running:
            self.pool.apply_async(
                self.exetute_requests_item_save, callback=self._callback)

    def run(self):
        self.get_url_list()

        for i in range(2):  # 控制并发
            self.pool.apply_async(
                self.exetute_requests_item_save, callback=self._callback)

        while True:  # 防止主线程结束
            time.sleep(0.0001)  # 避免cpu空转，浪费资源
            if self.total_response_num >= self.total_requests_num:
                self.is_running = False
                break


if __name__ == '__main__':
    t1 = time.time()
    tq_qiubai = TQQiuBai()
    tq_qiubai.run()
    print("total cost:", time.time()-t1)

{'user_name': 'BadG...', 'content': ['蓝天白云，嗨起来?']}
{'user_name': '白玉无尘', 'content': ['这学还能上？这孩子还能要？']}
{'user_name': '納森_陳...', 'content': ['谢谢，糊弄学更进一步了。']}
{'user_name': '爆笑大蒜...', 'content': ['我跟女友第一次约会结束后，我骑着自行车送她回家。经过她们村的一片玉米地时，女友就从自行车上蹦了下来，把我也拽停，扯着我的衣服就往玉米地里钻。我顿时激动万分，吓得']}
{'user_name': '一路高歌...', 'content': ['本..本人！？']}
{'user_name': 'PROX...', 'content': ['记得当时泰坦尼克号出发的时候，我拼尽全力失声呐喊：不要启航！但是没有人听我的，还骂我，最后我被电影院的保安架了出去。']}
{'user_name': '凉宫长门', 'content': ['哇:我疯狂期待！！！:啊！啊！啊！啊！逆天尖叫！']}
{'user_name': '巍人师表...', 'content': ['人类还有机会吗？']}
{'user_name': 'hawk...', 'content': ['#已经立秋了，告诉大家几个生活小常识：1、一天不能吃太多鸡蛋，吃太多，对母鸡身体不好，它来不及下。2、不要吃鸡屁股，鸡从来都不擦屁股。3、饭后吃水果是错误的观念']}
{'user_name': '一枕清霜', 'content': ['下面请听歌曲']}
{'user_name': '正儿八经...', 'content': ['模仿老公，看见老丈人来了VS老爸来了']}
{'user_name': '留恋那抹...', 'content': ['巩哥抻了一嘴的好面！😂']}
{'user_name': '东临～天...', 'content': ['教练还在医院，发誓出院后改行…！']}
{'user_name': '猪兔籽', 'content': ['功夫就系甘练成个']}
{'user_name': '骑着二哈...', 'content': ['江南皮革厂老板名字叫黄鹤其实还挺合理的，因为古代有个诗句是＂黄鹤一去不复返

#### 协程池实现爬虫

##### 协程池模块
```
import gevent.monkey
gevent.monkey.patch_all()
from gevent.pool import Pool
```

In [ ]:
import time
from queue import Queue
from lxml import etree
import requests
from gevent.pool import Pool
import gevent.monkey
gevent.monkey.patch_all()


class GQQiuBai:
    def __init__(self):
        self.url_temp = "https://www.qiushibaike.com/8hr/page/{}/"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36"
        }
        self.queue = Queue()
        self.pool = Pool(3)
        self.is_running = True
        self.total_requests_num = 0
        self.total_response_num = 0

    def get_url_list(self):  # 获取url列表
        for i in range(1, 14):
            self.queue.put(self.url_temp.format(i))
            self.total_requests_num += 1

    def parse_url(self, url):  # 发送请求，获取响应
        return requests.get(url, headers=self.headers).content.decode()

    def get_content_list(self, html_str):  # 提取段子
        html = etree.HTML(html_str)
        div_list = html.xpath('//*[@id="content"]/div/div[2]/div/ul/li')
        content_list = []
        for div in div_list:
            content = {}
            content["user_name"] = div.xpath(
                "./div/div/a/span/text()")[0].strip()
            content["content"] = [i.strip()
                                  for i in div.xpath("./div/a/text()")]
            content_list.append(content)
        return content_list

    def save_content_list(self, content_list):  # 保存数据
        for content in content_list:
            print(content)  # 此处对数据进行保存操作

    def exetute_requests_item_save(self):
        url = self.queue.get()
        html_str = self.parse_url(url)
        content_list = self.get_content_list(html_str)
        self.save_content_list(content_list)
        self.total_response_num += 1

    def _callback(self, temp):
        if self.is_running:
            self.pool.apply_async(
                self.exetute_requests_item_save, callback=self._callback)

    def run(self):
        self.get_url_list()

        for i in range(2):  # 控制并发
            self.pool.apply_async(
                self.exetute_requests_item_save, callback=self._callback)

        while True:  # 防止主线程结束
            time.sleep(0.0001)  # 避免cpu空转，浪费资源
            if self.total_response_num >= self.total_requests_num:
                self.is_running = False
                break


if __name__ == '__main__':
    t1 = time.time()
    gq_qiubai = GQQiuBai()
    gq_qiubai.run()
    print("total cost:", time.time()-t1)

## 招聘网站爬虫实战

- 在线JSON校验格式化工具：http://www.bejson.com/
- 招聘网站url：https://www.lagou.com/

In [28]:
import requests
from lxml import etree
url = 'https://www.lagou.com/jobs/positionAjax.json?city=%E5%8C%97%E4%BA%AC&needAddtionalResult=false'
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://www.lagou.com/jobs/list_%E8%BF%90%E7%BB%B4?city=%E6%88%90%E9%83%BD&cl=false&fromSearch=true&labelWords=&suginput=',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
}

s = requests.Session()
s.get(url='https://www.lagou.com/jobs/list_python%E5%85%A8%E6%A0%88?labelWords=&fromSearch=true&suginput=',
      headers=headers, timeout=3)
cookie = s.cookies
params = {
    "city": "北京",
    "needAddtionalResult": "false"
}

data = {
    "first": "true",
    "pn": "1",
    "kd": "python全栈",
}
res = requests.post(url, headers=headers, params=params,
                    data=data, cookies=cookie, timeout=3)
text = res.json()


In [38]:
all_data = text['content']['positionResult']['result']
for data in all_data:
    s = requests.Session()
    s.get(
        url='https://www.lagou.com/jobs/list_运维?city=北京&cl=false&fromSearch=true&labelWords=&suginput=',
        headers=headers, timeout=3)
    cookie1 = s.cookies
    url = 'https://www.lagou.com/jobs/' + \
        str(data.get('positionId')) + '.html'
    req1 = requests.get(url, headers=headers, cookies=cookie1)
    req1.encoding = 'utf-8'
    html = etree.HTML(req1.text)
    detail = ''.join(html.xpath('//*[@class="job-detail"]//*/text()')).strip()
    if detail.isspace():
        detail = ''.join(html.xpath('//*[@class="job-detail"]/text()')).strip()
        # print(detail)

    related_skills = data.get('skillLables')

    data_dict = {
        "firstType": str(data.get('firstType')),
        "secondType": str(data.get('secondType')),
        "thirdType": str(data.get('thirdType')),
        "city": str(data.get("city")),
        "positionName": str(data.get('positionName')),
        "district": str(data.get('district')),
        "stationname": str(data.get('stationname')),
        "jobNature": str(data.get('jobNature')),
        "companyLabelList": str(data.get('companyLabelList')),
        "industryField": str(data.get('industryField')),
        "salary": str(data.get('salary')),
        "companySize": str(data.get('companySize')),
        "skillLables": str(related_skills),
        "createTime": str(data.get('createTime')),
        "companyFullName": str(data.get('companyFullName')),
        "workYear": str(data.get('workYear')),
        "education": str(data.get('education')),
        "positionAdvantage": str(data.get('positionAdvantage')),
        "url": str(url),
        "detail": str(detail),
        "type": str(type),
        "latitude": str(data.get("latitude")),
        "longitude": str(data.get("longitude")),
#         "keyword": str(keyword),
        }
    print(data_dict)

{'firstType': '开发|测试|运维类', 'secondType': '后端开发', 'thirdType': 'Python', 'city': '北京', 'positionName': 'python全栈（ID:ZY）', 'district': '海淀区', 'stationname': '永丰南', 'jobNature': '全职', 'companyLabelList': '[]', 'industryField': '电商', 'salary': '8k-13k', 'companySize': '2000人以上', 'skillLables': "['Python']", 'createTime': '2020-09-03 17:19:48', 'companyFullName': '北京瑞友科技股份有限公司', 'workYear': '3-5年', 'education': '大专', 'positionAdvantage': '外企风格，中文工作环境、不加班、双休、节假日', 'url': 'https://www.lagou.com/jobs/7649378.html', 'detail': '', 'type': "<class 'type'>", 'latitude': '40.067787', 'longitude': '116.256382'}
{'firstType': '开发|测试|运维类', 'secondType': '后端开发', 'thirdType': 'Python', 'city': '上海', 'positionName': 'python全栈开发工程师', 'district': '浦东新区', 'stationname': '广兰路', 'jobNature': '全职', 'companyLabelList': '[]', 'industryField': '金融,软件开发', 'salary': '12k-24k', 'companySize': '50-150人', 'skillLables': "['Python', '全栈']", 'createTime': '2020-09-03 13:31:54', 'companyFullName': '上海淘利资产管理有限公司', 'workYe

{'firstType': '开发|测试|运维类', 'secondType': '后端开发', 'thirdType': 'C|C++', 'city': '杭州', 'positionName': 'C/C++,Python开发', 'district': 'None', 'stationname': 'None', 'jobNature': '全职', 'companyLabelList': "['福利完善', '国际化平台', '定期体检', '绩效奖金']", 'industryField': '通讯电子', 'salary': '12k-20k', 'companySize': '2000人以上', 'skillLables': "['C', 'C++', 'Python']", 'createTime': '2020-09-03 20:52:48', 'companyFullName': '华为技术有限公司', 'workYear': '1-3年', 'education': '本科', 'positionAdvantage': '薪资只要敢要就敢给，前景好，晋升快', 'url': 'https://www.lagou.com/jobs/7667979.html', 'detail': '', 'type': "<class 'type'>", 'latitude': 'None', 'longitude': 'None'}
{'firstType': '开发|测试|运维类', 'secondType': '后端开发', 'thirdType': 'Python', 'city': '北京', 'positionName': 'Python后端工程师', 'district': '海淀区', 'stationname': '知春路', 'jobNature': '全职', 'companyLabelList': "['节日礼物', '技能培训', '不限量零食', '岗位晋升']", 'industryField': '移动互联网,教育', 'salary': '20k-35k', 'companySize': '150-500人', 'skillLables': "['Python']", 'createTime': '2020-09-03 20:

大量数据散落在互联网中，要分析析互联网上的数据，需要先把数据从网络中获取下来，这就需要爬虫技术，在爬虫系统中，待抓取URL队例是很重要的一部分，待抓取URL队列中的URL以什么样的顺序排列也是一个很重要的问题，因为其决定了先抓取哪个页面、后抓取哪个页面。而决定这些URL排列顺序的方法叫做抓取策略，下面介绍，几种常见抓取策略：

  1.深度优先遍历策略，深度优先遍历策略是指网络爬虫会起始页开始，一个链接一个链接跟踪下云，处理完这条线路之后，再转入下一个起始页，继续跟踪链接。

  2.宽度优先遍历策略，宽度优先遍历策略的基本思路是将新下载网页中发现的链接直接插入待抓取URL队列的末尾，也就是说，网络爬虫会先抓取起始网页中链接的所有网页，然后再选择其中的一个链接网页，继续抓取网页中链接的所有网页。

  3.反向链接数策略，反向链接数是指一个网页被其他网页链接指向的数量。反向链接数表示的是一个网页的内容受到其他人推荐的程度。因此，很多时候搜索引擎的抓取系统会使用这个指标来评价网页的重要程度，从而决定不同网页抓取顺序。

  4.大站优先策略，对于待抓取URL队列中的所有网页，根据所属的网站进行分类；对于待下载页面数多的网站，则优先下载。这种策略也因此被叫作大站优先策略。

In [ ]:
import threading

import requests
from lxml import etree

from config import config
from utils.common import get_header
from utils.db_utils import insert
from collections import Counter


class LaGou(object):

    def __init__(self, thread=12):
        # self.keyword = keyword
        self.thread = thread
        # self.city = city
        # self.type = type
        self.baseurl = 'https://www.lagou.com/jobs/positionAjax.json'
        self.header = {
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Referer': 'https://www.lagou.com/jobs/list_%E8%BF%90%E7%BB%B4?city=%E6%88%90%E9%83%BD&cl=false&fromSearch=true&labelWords=&suginput=',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
        }

    def spider(self, keyword, city, type):

        expanded_skills = []

        max_page = 10
        for i in range(1, max_page):
            s = requests.Session()
            s.get(
                url='https://www.lagou.com/jobs/list_运维?city=北京&cl=false&fromSearch=true&labelWords=&suginput=',
                headers=get_header(), timeout=3)
            cookie = s.cookies
            res = requests.post(self.baseurl, headers=self.header, data={'first': True, 'pn': i, 'kd': keyword},
                                params={'px': 'default', 'city': city,
                                        'needAddtionalResult': 'false'},
                                cookies=cookie, timeout=3)
            text = res.json()
            all_data = text['content']['positionResult']['result']
            for data in all_data:
                s = requests.Session()
                s.get(
                    url='https://www.lagou.com/jobs/list_运维?city=北京&cl=false&fromSearch=true&labelWords=&suginput=',
                    headers=get_header(), timeout=3)
                cookie1 = s.cookies
                url = 'https://www.lagou.com/jobs/' + \
                    str(data.get('positionId')) + '.html'
                req1 = requests.get(url, headers=self.header, cookies=cookie1)
                req1.encoding = 'utf-8'
                html = etree.HTML(req1.text)
                detail = ''.join(html.xpath(
                    '//*[@class="job-detail"]//*/text()')).strip()
                if detail.isspace():
                    detail = ''.join(html.xpath(
                        '//*[@class="job-detail"]/text()')).strip()
                # print(detail)

                related_skills = data.get('skillLables')

                data_dict = {
                    "firstType": str(data.get('firstType')),
                    "secondType": str(data.get('secondType')),
                    "thirdType": str(data.get('thirdType')),
                    "city": str(data.get("city")),
                    "positionName": str(data.get('positionName')),
                    "district": str(data.get('district')),
                    "stationname": str(data.get('stationname')),
                    "jobNature": str(data.get('jobNature')),
                    "companyLabelList": str(data.get('companyLabelList')),
                    "industryField": str(data.get('industryField')),
                    "salary": str(data.get('salary')),
                    "companySize": str(data.get('companySize')),
                    "skillLables": str(related_skills),
                    "createTime": str(data.get('createTime')),
                    "companyFullName": str(data.get('companyFullName')),
                    "workYear": str(data.get('workYear')),
                    "education": str(data.get('education')),
                    "positionAdvantage": str(data.get('positionAdvantage')),
                    "url": str(url),
                    "detail": str(detail),
                    "type": str(type),
                    "latitude": str(data.get("latitude")),
                    "longitude": str(data.get("longitude")),
                    "keyword": str(keyword),
                }
                # print(data_dict)
                # time.sleep(random.randint(1, 5))

                expanded_skills += related_skills

                # print(related_skills)

                if not insert('lagou_copy1_copy1', **data_dict):
                    continue

        return [s.lower() for s in expanded_skills]

    def run(self):
        _, position, init_job = config()
        visited_jobs = set()
        thread_list = []
        for i in range(self.thread):
            t = threading.Thread(target=self.lagou_worker,
                                 args=(init_job, position, visited_jobs))
            thread_list.append(t)
        for t in thread_list:
            t.setDaemon(True)
            t.start()
        for t in thread_list:
            t.join()

    def lagou_worker(self, init_job, position, visited_jobs):

        while init_job:
            search_job = init_job.pop(0)

            print('We need to search {}, now search {}'.format(
                init_job, search_job))

            if search_job in visited_jobs:
                continue
            type = ''
            for k, v in position.items():
                if search_job in v:
                    type = k

            new_expaned = self.spider(search_job, '无锡', type)

            expaned_counter = Counter(new_expaned).most_common(n=5)

            new_jobs = [j for j, n in expaned_counter]

            init_job += new_jobs

            visited_jobs.add(search_job)


if __name__ == '__main__':

    LaGou().run()